In [3]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-7043.csv")

In [4]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)




#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
num_df=round(np.log(num_df.add(1)),2)
num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)


result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [5]:
X.shape

(7043, 19)

In [7]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
18,TotalCharges,1616.483021
15,Contract,1115.780167
9,OnlineSecurity,551.611529
12,TechSupport,523.303866
1,tenure,431.761590
10,OnlineBackup,230.086520
11,DeviceProtection,191.303140
5,Dependents,133.036443
16,PaperlessBilling,105.680863
0,SeniorCitizen,92.702566


In [8]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(7043, 15)

In [9]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3970 1204 504 1365
pod:  0.7303370786516854
pof:  0.23270197139543874
AUC:  0.7488175536281233


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits
{'var_smoothing': 3.5111917342151277e-07}
GaussianNB(var_smoothing=3.5111917342151277e-07)
0.7834247034754931


In [11]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=3.5111917342151277e-07)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4494 680 851 1018
pod:  0.5446762974852862
pof:  0.13142636258214146
AUC:  0.7066249674515723
accuracy:  0.78262104216953


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'C': 2, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=2, multi_class='ovr', penalty='l1', solver='liblinear')
0.8042031350741456


In [13]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4681 493 905 964
pod:  0.5157838416265382
pof:  0.09528411287205257
AUC:  0.7102498643772428
accuracy:  0.8015050404657106


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 10 folds for each of 16 candidates, totalling 160 fits
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}
KNeighborsClassifier(metric='manhattan', n_neighbors=19, weights='distance')
0.7557861863313992


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='distance', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4924 250 1470 399
pod:  0.21348314606741572
pof:  0.048318515655199075
AUC:  0.5825823152061084
accuracy:  0.7557858866960102


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 300}
RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
0.804343367182463


In [19]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4725 449 939 930
pod:  0.49759229534510435
pof:  0.08678005411673753
AUC:  0.7054061206141834
accuracy:  0.8029248899616641


In [20]:
# Decision Tree
import time
start = time.time()
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
{'criterion': 'entropy', 'splitter': 'random'}
DecisionTreeClassifier(criterion='entropy', splitter='random')
0.7282382333978079
Time:  5.2745537757873535


In [21]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier =DecisionTreeClassifier(criterion='entropy', splitter='random')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4228 946 936 933
pod:  0.49919743178170145
pof:  0.1828372632392733
AUC:  0.658180084271214
accuracy:  0.7327843248615646


In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.24545454545454548, 'n_estimators': 10, 'subsample': 0.95}
GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.24545454545454548,
                           n_estimators=10, subsample=0.95)
0.7942627740167634


In [23]:
#  Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier =GradientBoostingClassifier(learning_rate=0.2, max_depth=5, max_features='sqrt',
                           min_samples_leaf=0.1,
                           min_samples_split=0.24545454545454548,
                           n_estimators=10, subsample=0.95)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4887 287 1195 674
pod:  0.3606206527554842
pof:  0.05546965597216853
AUC:  0.6525754983916578
accuracy:  0.7895783046997018


In [25]:
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    n_cols = x_train_chi.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        n_cols = x_train_chi.shape[1]
        nn = Sequential()
        nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        nn.add(Dense(250, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return nn
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

Best: 0.789152 using {'batch_size': 20, 'epochs': 100}
0.645715 (0.163262) with: {'batch_size': 10, 'epochs': 10}
0.788727 (0.002990) with: {'batch_size': 10, 'epochs': 50}
0.780489 (0.010671) with: {'batch_size': 10, 'epochs': 100}
0.751381 (0.026364) with: {'batch_size': 20, 'epochs': 10}
0.774667 (0.012816) with: {'batch_size': 20, 'epochs': 50}
0.789152 (0.002452) with: {'batch_size': 20, 'epochs': 100}
0.590677 (0.200876) with: {'batch_size': 40, 'epochs': 10}
0.730940 (0.040561) with: {'batch_size': 40, 'epochs': 50}
0.775525 (0.015329) with: {'batch_size': 40, 'epochs': 100}
0.604448 (0.181702) with: {'batch_size': 60, 'epochs': 10}
0.752233 (0.014308) with: {'batch_size': 60, 'epochs': 50}
0.759333 (0.019736) with: {'batch_size': 60, 'epochs': 100}
0.734487 (0.004485) with: {'batch_size': 80, 'epochs': 10}
0.750104 (0.018042) with: {'batch_size': 80, 'epochs': 50}
0.740740 (0.055637) with: {'batch_size': 80, 'epochs': 100}
0.734629 (0.004602) with: {'batch_size': 100, 'epochs':

In [29]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

import time
start = time.time()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, batch_size= 20, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)


end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/30
254/254 [==============================] - 2s 5ms/step - loss: 12.1402 - accuracy: 0.5966 - val_loss: 1.4526 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 3ms/step - loss: 0.9926 - accuracy: 0.6336 - val_loss: 0.5935 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 3ms/step - loss: 0.6879 - accuracy: 0.7124 - val_loss: 0.5919 - val_accuracy: 0.7279
Epoch 4/30
254/254 [==============================] - 1s 3ms/step - loss: 0.6181 - accuracy: 0.7227 - val_loss: 0.5967 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 3ms/step - loss: 0.5890 - accuracy: 0.7372 - val_loss: 0.5894 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 3ms/step - loss: 0.5874 - accuracy: 0.7288 - val_loss: 0.5841 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 3ms/step - loss: 0.5889 - accuracy: 0.7240 - val_loss: 0.5839 - val_accuracy: 0.7279
Epoch

254/254 [==============================] - 2s 5ms/step - loss: 5.4229 - accuracy: 0.6114 - val_loss: 0.7082 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 0.7801 - accuracy: 0.6692 - val_loss: 0.7647 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6382 - accuracy: 0.7198 - val_loss: 0.5811 - val_accuracy: 0.7279
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5994 - accuracy: 0.7209 - val_loss: 0.6304 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5787 - accuracy: 0.7249 - val_loss: 0.5643 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5856 - accuracy: 0.7147 - val_loss: 0.6263 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5892 - accuracy: 0.7176 - val_loss: 0.5465 - val_accuracy: 0.7279
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 9.7749 - accuracy: 0.6101 - val_loss: 2.2647 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 1.4249 - accuracy: 0.6347 - val_loss: 0.6929 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6774 - accuracy: 0.6876 - val_loss: 0.8475 - val_accuracy: 0.7279
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6733 - accuracy: 0.6991 - val_loss: 0.6715 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6113 - accuracy: 0.7225 - val_loss: 0.6139 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5919 - accuracy: 0.7327 - val_loss: 0.5938 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 3ms/step - loss: 0.5948 - accuracy: 0.7235 - val_loss: 0.5913 - val_accuracy: 0.7287
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 9.1613 - accuracy: 0.6161 - val_loss: 0.6601 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 0.7481 - accuracy: 0.6755 - val_loss: 0.5965 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6344 - accuracy: 0.7193 - val_loss: 0.6266 - val_accuracy: 0.7287
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6047 - accuracy: 0.7330 - val_loss: 0.5888 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5803 - accuracy: 0.7415 - val_loss: 0.5884 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5824 - accuracy: 0.7346 - val_loss: 0.5803 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5772 - accuracy: 0.7350 - val_loss: 0.5819 - val_accuracy: 0.7279
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 5.2975 - accuracy: 0.6212 - val_loss: 0.6103 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6839 - accuracy: 0.6970 - val_loss: 0.5892 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5972 - accuracy: 0.7358 - val_loss: 0.5911 - val_accuracy: 0.7279
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5852 - accuracy: 0.7372 - val_loss: 0.5990 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5816 - accuracy: 0.7367 - val_loss: 0.5830 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5776 - accuracy: 0.7369 - val_loss: 0.5808 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5715 - accuracy: 0.7393 - val_loss: 0.5920 - val_accuracy: 0.7279
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 15.8002 - accuracy: 0.6007 - val_loss: 2.8451 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 1.3210 - accuracy: 0.6304 - val_loss: 0.6214 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6617 - accuracy: 0.7230 - val_loss: 0.6166 - val_accuracy: 0.7279
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6232 - accuracy: 0.7288 - val_loss: 0.5870 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6241 - accuracy: 0.7228 - val_loss: 0.5868 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5749 - accuracy: 0.7397 - val_loss: 0.5905 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5754 - accuracy: 0.7405 - val_loss: 0.5829 - val_accuracy: 0.7279
Epoch 8/30
254/2

254/254 [==============================] - 2s 5ms/step - loss: 12.5515 - accuracy: 0.6027 - val_loss: 1.2075 - val_accuracy: 0.2886
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 0.9388 - accuracy: 0.6350 - val_loss: 0.5999 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6422 - accuracy: 0.7227 - val_loss: 0.9849 - val_accuracy: 0.3565
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6881 - accuracy: 0.6891 - val_loss: 0.6054 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5860 - accuracy: 0.7317 - val_loss: 0.5891 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5821 - accuracy: 0.7366 - val_loss: 0.6183 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6083 - accuracy: 0.7344 - val_loss: 0.5827 - val_accuracy: 0.7279
Epoch 8/30
254/2

254/254 [==============================] - 2s 5ms/step - loss: 7.5442 - accuracy: 0.5965 - val_loss: 2.1688 - val_accuracy: 0.7279
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 0.8232 - accuracy: 0.6625 - val_loss: 0.6069 - val_accuracy: 0.7279
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5996 - accuracy: 0.7334 - val_loss: 0.8324 - val_accuracy: 0.4669
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6095 - accuracy: 0.7167 - val_loss: 0.5135 - val_accuracy: 0.7279
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5507 - accuracy: 0.7318 - val_loss: 0.6265 - val_accuracy: 0.7279
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5211 - accuracy: 0.7453 - val_loss: 0.5142 - val_accuracy: 0.7279
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5364 - accuracy: 0.7340 - val_loss: 0.4841 - val_accuracy: 0.7666
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 9.8149 - accuracy: 0.6033 - val_loss: 1.3306 - val_accuracy: 0.7248
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 1.1601 - accuracy: 0.6287 - val_loss: 0.6236 - val_accuracy: 0.7248
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.7973 - accuracy: 0.6744 - val_loss: 0.7033 - val_accuracy: 0.4740
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6763 - accuracy: 0.6872 - val_loss: 0.6010 - val_accuracy: 0.7248
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6430 - accuracy: 0.7055 - val_loss: 0.6003 - val_accuracy: 0.7248
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5951 - accuracy: 0.7382 - val_loss: 0.5765 - val_accuracy: 0.7256
Epoch 7/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5957 - accuracy: 0.7235 - val_loss: 0.6365 - val_accuracy: 0.7248
Epoch 8/30
254/25

254/254 [==============================] - 2s 5ms/step - loss: 12.8423 - accuracy: 0.5997 - val_loss: 0.6354 - val_accuracy: 0.7248
Epoch 2/30
254/254 [==============================] - 1s 4ms/step - loss: 1.4982 - accuracy: 0.6175 - val_loss: 0.6402 - val_accuracy: 0.7248
Epoch 3/30
254/254 [==============================] - 1s 4ms/step - loss: 0.7092 - accuracy: 0.7040 - val_loss: 0.5853 - val_accuracy: 0.7248
Epoch 4/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6379 - accuracy: 0.7141 - val_loss: 0.5888 - val_accuracy: 0.7248
Epoch 5/30
254/254 [==============================] - 1s 4ms/step - loss: 0.6503 - accuracy: 0.7096 - val_loss: 0.6135 - val_accuracy: 0.7248
Epoch 6/30
254/254 [==============================] - 1s 4ms/step - loss: 0.5749 - accuracy: 0.7420 - val_loss: 0.5862 - val_accuracy: 0.7240
y2_pred:  [[3.08836609e-01]
 [3.09573531e-01]
 [2.63866246e-01]
 [2.65433311e-01]
 [2.59274542e-01]
 [2.90159047e-01]
 [2.76874244e-01]
 [3.10823023e-01]
 [3.

In [27]:
##RNN Hyperparemeter tuning
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    #n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        #n_cols = X2_train.shape[1]
        #nn = Sequential()
        #nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        #nn.add(Dense(250, activation='relu'))
        #nn.add(Dense(1, activation='sigmoid'))
        #nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model = Sequential()
        model.add(Embedding(10000, 32))
        model.add(LSTM(32))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
        return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

Best: 0.755785 using {'batch_size': 100, 'epochs': 10}
0.659384 (0.038112) with: {'batch_size': 10, 'epochs': 10}
0.691604 (0.032324) with: {'batch_size': 10, 'epochs': 50}
0.702397 (0.014011) with: {'batch_size': 10, 'epochs': 100}
0.676270 (0.023021) with: {'batch_size': 20, 'epochs': 10}
0.686211 (0.022870) with: {'batch_size': 20, 'epochs': 50}
0.701686 (0.018773) with: {'batch_size': 20, 'epochs': 100}
0.676414 (0.015882) with: {'batch_size': 40, 'epochs': 10}
0.694445 (0.040015) with: {'batch_size': 40, 'epochs': 50}
0.713044 (0.020043) with: {'batch_size': 40, 'epochs': 100}
0.744001 (0.003175) with: {'batch_size': 60, 'epochs': 10}
0.714322 (0.035280) with: {'batch_size': 60, 'epochs': 50}
0.708218 (0.018841) with: {'batch_size': 60, 'epochs': 100}
0.748827 (0.008448) with: {'batch_size': 80, 'epochs': 10}
0.725687 (0.012475) with: {'batch_size': 80, 'epochs': 50}
0.723409 (0.027189) with: {'batch_size': 80, 'epochs': 100}
0.755785 (0.004701) with: {'batch_size': 100, 'epochs':

In [28]:
#RNN
import time
start = time.time()

def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=10, batch_size=100, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6056 - acc: 0.6926 - val_loss: 0.5213 - val_acc: 0.7350
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5141 - acc: 0.7370 - val_loss: 0.5091 - val_acc: 0.7461
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4883 - acc: 0.7597 - val_loss: 0.5028 - val_acc: 0.7429
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4883 - acc: 0.7648 - val_loss: 0.5021 - val_acc: 0.7555
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4752 - acc: 0.7629 - val_loss: 0.4965 - val_acc: 0.7571
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4613 - acc: 0.7845 - val_loss: 0.4964 - val_acc: 0.7571
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4414 - acc: 0.7902 - val_loss: 0.4973 - val_acc: 0.7571
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4099 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6090 - acc: 0.7169 - val_loss: 0.5300 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5135 - acc: 0.7372 - val_loss: 0.5085 - val_acc: 0.7484
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4979 - acc: 0.7516 - val_loss: 0.5050 - val_acc: 0.7445
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4861 - acc: 0.7614 - val_loss: 0.5014 - val_acc: 0.7532
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4768 - acc: 0.7686 - val_loss: 0.4993 - val_acc: 0.7539
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4511 - acc: 0.7840 - val_loss: 0.5012 - val_acc: 0.7539
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4339 - acc: 0.7955 - val_loss: 0.4967 - val_acc: 0.7587
Epoch 8/10
51/51 [==============================] - 1s 17ms/step - loss: 0.3910 - a

Epoch 1/10
51/51 [==============================] - 4s 29ms/step - loss: 0.6034 - acc: 0.7057 - val_loss: 0.5183 - val_acc: 0.7366
Epoch 2/10
51/51 [==============================] - 1s 18ms/step - loss: 0.5162 - acc: 0.7381 - val_loss: 0.5055 - val_acc: 0.7524
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4855 - acc: 0.7703 - val_loss: 0.5022 - val_acc: 0.7555
Epoch 4/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4851 - acc: 0.7670 - val_loss: 0.4962 - val_acc: 0.7595
Epoch 5/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4807 - acc: 0.7706 - val_loss: 0.4938 - val_acc: 0.7634
Epoch 6/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4574 - acc: 0.7862 - val_loss: 0.5135 - val_acc: 0.7626
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4314 - acc: 0.8029 - val_loss: 0.4948 - val_acc: 0.7595
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4090 - a

Epoch 1/10
51/51 [==============================] - 4s 29ms/step - loss: 0.6002 - acc: 0.6941 - val_loss: 0.5217 - val_acc: 0.7366
Epoch 2/10
51/51 [==============================] - 1s 19ms/step - loss: 0.5161 - acc: 0.7378 - val_loss: 0.5085 - val_acc: 0.7476
Epoch 3/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4799 - acc: 0.7694 - val_loss: 0.5053 - val_acc: 0.7429
Epoch 4/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4748 - acc: 0.7729 - val_loss: 0.4983 - val_acc: 0.7524
Epoch 5/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4737 - acc: 0.7693 - val_loss: 0.4949 - val_acc: 0.7555
Epoch 6/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4474 - acc: 0.7881 - val_loss: 0.4920 - val_acc: 0.7610
Epoch 7/10
51/51 [==============================] - 1s 17ms/step - loss: 0.4191 - acc: 0.8133 - val_loss: 0.4943 - val_acc: 0.7618
Epoch 8/10
51/51 [==============================] - 1s 17ms/step - loss: 0.3826 - a

Epoch 1/10
51/51 [==============================] - 4s 30ms/step - loss: 0.6055 - acc: 0.7096 - val_loss: 0.5204 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 19ms/step - loss: 0.5072 - acc: 0.7390 - val_loss: 0.5084 - val_acc: 0.7579
Epoch 3/10
51/51 [==============================] - 1s 20ms/step - loss: 0.4761 - acc: 0.7654 - val_loss: 0.5071 - val_acc: 0.7508
Epoch 4/10
51/51 [==============================] - 1s 20ms/step - loss: 0.4756 - acc: 0.7682 - val_loss: 0.5094 - val_acc: 0.7524
Epoch 5/10
51/51 [==============================] - 1s 20ms/step - loss: 0.4737 - acc: 0.7702 - val_loss: 0.4971 - val_acc: 0.7500
Epoch 6/10
51/51 [==============================] - 1s 20ms/step - loss: 0.4397 - acc: 0.7884 - val_loss: 0.5011 - val_acc: 0.7658
Epoch 7/10
51/51 [==============================] - 1s 19ms/step - loss: 0.4258 - acc: 0.8055 - val_loss: 0.4946 - val_acc: 0.7555
Epoch 8/10
51/51 [==============================] - 1s 18ms/step - loss: 0.3963 - a